# Create an AI Helper to Automate Filling Boring HTML Forms

### Audience

This article is intended for programmers who want to learn about the basics of "function calling" capability of Large Language Models (LLMs) (such as [OpenAI](https://openai.com/) or [Anthropic](https://www.anthropic.com/)). Function calling is a fundamental LLM feature that allows creating specialized tools, agents, or assistants that can interact with the external world. This article will show you how to create a simple AI assistant that can fill out HTML forms.

### What You Will Learn

Function calling is enabled by providing the LLM with a definition of the function signature. The function signature is a description of the function's expected input properties. You will learn how to create a dynamically generated [JSON schema](https://json-schema.org/) function signature, allowing the AI assistant to interact with HTML forms. [JSON schema](https://json-schema.org/) is a powerful tool for defining and validating the structure of a JSON object. For the educational purpose will not use any external libraries, only pure JavaScript code.

### Introduction

We can all agree that filling out forms is a boring and time-consuming task. What if we could create an AI assistant that could fill the form for us, allowing us to dedicate our time to more constructive tasks?

The AI assistant will be able to fill the form by calling a function with the form fields as arguments. The function will return a JSON object with the form fields as keys and the values to be filled in the form.

There are an infinite number of forms on the web, each with its own unique structure and naming conventions. Until recently, it was almost impossible to create a generic assistant that could fill any form. But with the advent of LLMs, we can create one.

Function calling in most scenarios allows LLMs to interact with [APIs](https://en.wikipedia.org/wiki/API), but the vast majority of web applications do not expose APIs, and the only way to interact with them is by filling out forms.

### Let's Get Started

Forms can be very different from each other, but they all are built using common elements like input fields, textareas, checkboxes, radio buttons, etc.

First, we need to identify the form elements and their types. Regardless of the element type, each element is expected to have a "name" attribute that will be later used as a key in the JSON object.

For each element type, we will create a function that will return a fragment of [JSON schema](https://json-schema.org/) defining the element. The [JSON schema](https://json-schema.org/) should contain a description of the element's purpose. This is very useful for LLMs to understand the purpose of the element or expected values. The text for the description will be gathered from the element's label or placeholder attribute.

First we will define some utility functions that will be used to extract element description and format element name, and group elements by name.

```javascript
const getElementDescription = (element) => {
  const labelsText = Array.from(element.labels)
    .map((label) => label.innerText)
    .join(', ');
  return `${labelsText} ${element.placeholder || ''}`.trim();
};

const formatName = (name) => name.replace(/[^a-zA-Z0-9_-]/g, '_').slice(0, 63);

const getDescription = (element) => {
  const describeId = element.getAttribute('aria-describedby');
  return describeId ? document.querySelector(`#${describeId}`).innerText : '';
};

const groupByName = (arr) =>
  Object.entries(
    arr.reduce((result, obj) => {
      const { name, value, id } = obj;
      if (!result[name]) result[name] = [];
      result[name].push({
        const: value,
        id,
        title: getElementDescription(obj),
      });
      return result;
    }, {})
  );
```

Here we define functions to create schema for input, select, textarea, checkboxes and radios.

Json schema for each element at least contains:
- `name`: element's name
- `type`: element type, usually string
- `description`: element description

Than we can have more additional fields depending on element type. For example, in case of input element we can have `min`, `max`, `pattern` and `required` fields.
In case of select element, radio or checkbox elements we also add enum field with all possible values. Particular checkboxes and radio elements should be addressed in special way since they can have multiple values options related to one name.

```javascript
const getInputSchema = (input) => {
  const { name, type, min, max, pattern, required } = input;
  if (!name) return null;

  const schema = {
    name,
    type: type === 'number' ? 'number' : 'string',
    description: getElementDescription(input),
  };

  if (min) schema.minimum = Number(min);
  if (max) schema.maximum = Number(max);
  if (pattern) schema.pattern = pattern;

  return [formatName(name), schema, required];
};

const getSelectSchema = (select) => {
  const { name, required } = select;
  if (!name) return null;

  return [
    formatName(name),
    {
      name,
      type: 'string',
      description: getElementDescription(select),
      enum: Array.from(select.options).map((option) => option.value),
    },
    required,
  ];
};

const getTextareaSchema = (textArea) => {
  const { name, required } = textArea;
  if (!name) return null;

  return [
    formatName(name),
    { name, type: 'string', description: getElementDescription(textArea) },
    required,
  ];
};

const getCheckboxesSchema = ([name, values]) => {
  const element = document.querySelector(`[name="${name}"]`);
  const isArray = name.endsWith('[]');

  const schema = {
    name,
    type: isArray ? 'array' : 'boolean',
    description: getDescription(element),
  };

  if (isArray) {
    schema.uniqueItems = true;
    schema.items = { oneOf: values };
  }

  return [formatName(name), schema];
};

const getRadioSchema = ([name, values]) => {
  const element = document.querySelector(`[name="${name}"]`);
  return [
    formatName(name),
    {
      name,
      type: 'string',
      description: getDescription(element),
      enum: values.map((v) => v.const),
    },
  ];
};
```

Now we arrive to the most important function that will generate the schema for each form using all available functions.

```javascript
const generateSchema = (form) => {
  const inputSelectors = [
    'input[type="text"]',
    'input[type="email"]',
    'input[type="number"]',
    'input[type="password"]',
    'input[type="tel"]',
    'input[type="url"]',
    'input[type="date"]',
    'input[type="time"]',
    'input[type="datetime-local"]',
    'input[type="month"]',
    'input[type="week"]',
    'input[type="color"]',
    'input[type="range"]',
    'input[type="search"]',
  ].join(', ');

  const inputs = Array.from(form.querySelectorAll(inputSelectors))
    .map(getInputSchema)
    .filter(Boolean);
  const checkboxes = groupByName(
    Array.from(form.querySelectorAll('input[type="checkbox"]'))
  ).map(getCheckboxesSchema);
  const radios = groupByName(
    Array.from(form.querySelectorAll('input[type="radio"]'))
  ).map(getRadioSchema);
  const selects = Array.from(form.getElementsByTagName('select'))
    .map(getSelectSchema)
    .filter(Boolean);
  const textAreas = Array.from(form.getElementsByTagName('textarea'))
    .map(getTextareaSchema)
    .filter(Boolean);

  const schemaProps = [
    ...inputs,
    ...checkboxes,
    ...radios,
    ...selects,
    ...textAreas,
  ];
  const required = schemaProps.filter(([, , r]) => r).map(([name]) => name);

  return {
    name: 'fillup_form',
    description: 'Schema to fill form inputs',
    parameters: {
      type: 'object',
      required,
      properties: Object.fromEntries(
        schemaProps.map(([name, schema]) => [name, schema])
      ),
    },
  };
};
```

This function scans the form for all input elements and creates a schema for each of them. It groups checkboxes and radios by name and creates a schema for each group. Finally, it creates a JSON schema with all the form elements.

`fillForm` populates the form with the data provided in the JSON object. The function iterates over the JSON object and fill the form fields with corresponding values.

```javascript
const fillForm = (formFields, inputData) => {
  inputData.forEach(([name, value]) => {
    try {
      const fieldDef = formFields[name];
      const fieldName = fieldDef.name;
      const fieldElement = document.querySelector(`[name="${fieldName}"]`);

      if (Array.isArray(value)) {
        value.forEach((val) => {
          const checkbox = document.querySelector(
            `[name="${fieldName}"][value="${val}"]`
          );
          if (checkbox) checkbox.checked = true;
        });
      } else if (fieldElement.type === 'radio') {
        const radio = document.querySelector(
          `[name="${fieldName}"][value="${value}"]`
        );
        if (radio) radio.checked = true;
      } else if (fieldElement) {
        fieldElement.value = value;
      }
    } catch (error) {
      console.error(`Error filling form field: ${name}`, error);
    }
  });
};
```

Here we define a function that will call OpenAI API with the provided parameters.

```javascript
const callOpenAiAPI = async ({
  api_key,
  model = 'gpt-4o',
  max_tokens = 3024,
  tools,
  messages,
}) => {
  try {
    const response = await fetch('https://api.openai.com/v1/chat/completions', {
      method: 'POST',
      headers: {
        Authorization: `Bearer ${api_key}`,
        'Content-Type': 'application/json',
      },
      body: JSON.stringify({
        model,
        max_tokens,
        temperature: 0,
        tools,
        messages,
      }),
    });
    return await response.json();
  } catch (error) {
    console.error('Error calling OpenAI API:', error);
    throw error;
  }
};
```

This function takes provided by OpenAi object and populates the form with the with values.

```javascript
const fillForm = (formFields, inputData) => {
  inputData.forEach(([name, value]) => {
    try {
      const fieldDef = formFields[name];
      const fieldName = fieldDef.name;
      const fieldElement = document.querySelector(`[name="${fieldName}"]`);

      if (Array.isArray(value)) {
        value.forEach((val) => {
          const checkbox = document.querySelector(
            `[name="${fieldName}"][value="${val}"]`
          );
          if (checkbox) checkbox.checked = true;
        });
      } else if (fieldElement.type === 'radio') {
        const radio = document.querySelector(
          `[name="${fieldName}"][value="${value}"]`
        );
        if (radio) radio.checked = true;
      } else if (fieldElement) {
        fieldElement.value = value;
      }
    } catch (error) {
      console.error(`Error filling form field: ${name}`, error);
    }
  });
};
```

`setupForms` scans html document for all forms and creates a dialog for each form. The dialog contains a textarea for the user to input the data and an input field for the OpenAI API key. The dialog also contains a "Fill" button that, when clicked, will open the dialog and fill the form with the data provided by the user.

```javascript
const setupForms = () => {
  const forms = Array.from(document.getElementsByTagName('form'));

  forms.forEach((form, i) => {
    const dialogHtml = `
      <a class="fill_btn" onclick="document.getElementById('dialog_${i}').showModal()">✨ Fill</a>
      <dialog class="fill" id="dialog_${i}">
        <article>
          <p>
            <textarea class="fill" placeholder="Your data" id="_data_${i}" required></textarea>
          </p>
          <p>
            <input type="password" placeholder="OpenAI API key" id="_api-key_${i}" required />
          </p>
          <p style="text-align: right">
            <button role="button" onclick="dialog_${i}.close()">Close</button>
            <button role="button" id="btn-submit-${i}" onclick="dialog_${i}.close()">Submit</button>
          </p>
        </article>
      </dialog>
     <style>
dialog.fill::backdrop {
  background: black;
  opacity: 0.7;
}

dialog.fill{
  border-radius: .3em;
  article {
    background: white;
    width: 40em;
  }
  textarea {
    width: 100%;
    height: 6em;
  }
  input {
    width: 100%;
  }
}
.fill_btn {
  cursor: pointer;
  margin-left: .5em;
  color: yellow;
  background: black;
  width: 6em;
  height: 100%;
  display: flex;
  align-content: center;
  justify-content: center;
  align-items: center;
  font-weight: bold;
  text-align: center;
}
@keyframes spinner {
  to {transform: rotate(360deg);}
}
 
.spinner:before {
  content: '';
  box-sizing: border-box;
  position: absolute;
  top: 50%;
  left: 50%;
  width: 20px;
  height: 20px;
  margin-top: -10px;
  margin-left: -10px;
  border-radius: 50%;
  border: 2px solid #ccc;
  border-top-color: #000;
  animation: spinner .6s linear infinite;
}
</style> 
    `;

    const dialogContainer = document.createElement('div');
    dialogContainer.innerHTML = dialogHtml;
    const submitButton = form.querySelector('button[type="submit"]');
    submitButton.parentElement.appendChild(dialogContainer);

    document
      .getElementById(`btn-submit-${i}`)
      .addEventListener('click', () => submitForm(submitButton, form, i));
  });
};
```


Finally we got to the point where we can call the OpenAI API to fill the form with fake data. We will create a button that will trigger the process of filling the form.

```javascript

const forms = Array.from(document.getElementsByTagName('form'));

forms.forEach((form) => {
  const button = document.createElement('a');
  button.onclick = async () => {
    const formSchema = generateSchema(form);
    const llm = await callOpenAiAPI({
      api_key,
      tools: [
        {
          type: 'function',
          function: formSchema,
        },
      ],
      tool_choice: 'auto',
      response_format: { type: 'json_object' },
      messages: [{ role: 'user', content: '`fillup_form` with fake data. No additional comments. only json data' }],
    });

    try {
      const rawData = llm.choices[0].message?.tool_calls[0].function.arguments;
      const inputData = Object.entries(JSON.parse(rawData));
      fillForm(formSchema.parameters.properties, inputData);
    } catch (error) {
      const rawData = llm.choices[0].message.content;
      const inputData = Object.entries(JSON.parse(rawData));
      fillForm(formSchema.parameters.properties, inputData);
    }
  };
  button.innerText = 'Fill Form';
  button.style = 'position: absolute; top: 0; right: 0;';
  form.appendChild(button);
});
```

First we iterate all forms on the page and for each form we create a button that will trigger the process of filling the form. When the button is clicked we generate the schema for the form and call the OpenAI API to fill the form with fake data. The response from the API is then used to fill the form.


### Where we can go with it?

This script can be further improved by adding more advanced features like handling file uploads, handling dynamic forms, etc.
Possible further developments:

- `browser extension` that will automatically fill the forms on the page based on stored profiles.

- use LLM to generate `generate fake data` in scenarios where we need to preserve out privacy.

- We can extend script with some persistance layer to store filled values so we can use them in future.

- In case of very complex forms with big number of fields brake the form into smaller parts (fieldsets) and fill them separately.

### Conclusion

This script should work with most forms. However if form is dynamic (some form elements are change or activated on user input) or uses some advanced features like file upload or is built in non standard or erroneous way it may not work as expected. In such cases you may need to adjust the script to handle these cases.

I hope you found this article useful and that you learned something new. If you have any questions or suggestions, feel free to leave a comment below.


Student Name: Emily Johnson
Student ID: 12345
Email: emily@example.com
Birthday: 04/12/1998
Address: 1234 Elm Street, Springfield, IL 62701
Phone: 217 8123438
Extra Curricular Activities: Swimming, Reading, Writing
Skills/Talents: Acting, Dancing, Singing
Sports: Tennis, Soccer, Chess
i Have Any Scholarship
i want to work after collage


// https://www.jotform.com/form-templates/preview/90722179146964/classic&nofs&disableSmartEmbed=1

// https://www.jotform.com/form-templates/registration/student-registration-forms

// https://jsonforms.io/docs/multiple-choice/

// TODO use aria-describedby="label_25" to get label


{  
  "q4_fullName_first_": "Emily",  
  "q4_fullName_last_": "Johnson",  
  "q9_studentNumber9": "12345",  
  "q10_email": "emily@example.com",  
  "q15_birthDate_month_": "4",  
  "q15_birthDate_day_": "12",  
  "q15_birthDate_year_": "1998",  
  "q44_presentAddress44_addr_line1_": "1234 Elm Street",  
  "q44_presentAddress44_city_": "Springfield",  
  "q44_presentAddress44_state_": "IL",  
  "q44_presentAddress44_postal_": "62701",  
  "q61_phoneNumber_full_": "217 8123438",  
  "q26_extraCurricular__": ["Swimming", "Reading", "Writing"],  
  "q25_skillstalents__": ["acting", "dancing", "singing"],  
  "q28_sports__": ["Tennis", "Soccer", "Chess"],  
  "q48_suggestions48": ""
}

In [ ]:
{  "q4_fullName_first_": "Emily",  "q4_fullName_last_": "Johnson",  "q9_studentNumber9": "12345",  "q10_email": "emily@example.com",  "q15_birthDate_month_": "4",  "q15_birthDate_day_": "12",  "q15_birthDate_year_": "1998",  "q44_presentAddress44_addr_line1_": "1234 Elm Street",  "q44_presentAddress44_city_": "Springfield",  "q44_presentAddress44_state_": "IL",  "q44_presentAddress44_postal_": "62701",  "q61_phoneNumber_full_": "217 8123438",  "q26_extraCurricular__": ["Swimming", "Reading", "Writing"],  "q25_skillstalents__": ["acting", "dancing", "singing"],  "q28_sports__": ["Tennis", "Soccer", "Chess"],  "q48_suggestions48": ""}


In [ ]:
{
  "name": "fillup_form",
  "description": "Schema to fill form inputs",
  "parameters": {
    "type": "object",
    "required": [
      "q44_presentAddress44_addr_line1_",
      "q44_presentAddress44_city_",
      "q44_presentAddress44_state_",
      "q44_presentAddress44_postal_",
      "q45_permanentAddress45_addr_line1_",
      "q45_permanentAddress45_city_",
      "q45_permanentAddress45_state_",
      "q45_permanentAddress45_postal_",
      "q63_addressOf63_addr_line1_",
      "q63_addressOf63_city_",
      "q63_addressOf63_state_",
      "q63_addressOf63_postal_",
      "q44_presentAddress44_country_",
      "q45_permanentAddress45_country_",
      "_data"
    ],
    "properties": {
      "q4_fullName_first_": {
        "type": "string",
        "name": "q4_fullName[first]",
        "description": "Full Name, First Name"
      },
      "q4_fullName_middle_": {
        "type": "string",
        "name": "q4_fullName[middle]",
        "description": "Middle Name"
      },
      "q4_fullName_last_": {
        "type": "string",
        "name": "q4_fullName[last]",
        "description": "Last Name"
      },
      "q9_studentNumber9": {
        "type": "string",
        "name": "q9_studentNumber9",
        "description": "Student Number, e.g. 11183021"
      },
      "q10_email": {
        "type": "string",
        "name": "q10_email",
        "description": "Email ex: myname@example.com"
      },
      "q61_phoneNumber_full_": {
        "type": "string",
        "name": "q61_phoneNumber[full]",
        "description": "Phone Number (000) 000-0000"
      },
      "q44_presentAddress44_addr_line1_": {
        "type": "string",
        "name": "q44_presentAddress44[addr_line1]",
        "description": "Present Address, Street Address"
      },
      "q44_presentAddress44_addr_line2_": {
        "type": "string",
        "name": "q44_presentAddress44[addr_line2]",
        "description": "Street Address Line 2"
      },
      "q44_presentAddress44_city_": {
        "type": "string",
        "name": "q44_presentAddress44[city]",
        "description": "City"
      },
      "q44_presentAddress44_state_": {
        "type": "string",
        "name": "q44_presentAddress44[state]",
        "description": "State / Province"
      },
      "q44_presentAddress44_postal_": {
        "type": "string",
        "name": "q44_presentAddress44[postal]",
        "description": "Postal / Zip Code"
      },
      "q45_permanentAddress45_addr_line1_": {
        "type": "string",
        "name": "q45_permanentAddress45[addr_line1]",
        "description": "Permanent Address, Street Address"
      },
      "q45_permanentAddress45_addr_line2_": {
        "type": "string",
        "name": "q45_permanentAddress45[addr_line2]",
        "description": "Street Address Line 2"
      },
      "q45_permanentAddress45_city_": {
        "type": "string",
        "name": "q45_permanentAddress45[city]",
        "description": "City"
      },
      "q45_permanentAddress45_state_": {
        "type": "string",
        "name": "q45_permanentAddress45[state]",
        "description": "State / Province"
      },
      "q45_permanentAddress45_postal_": {
        "type": "string",
        "name": "q45_permanentAddress45[postal]",
        "description": "Postal / Zip Code"
      },
      "q62_personTo62_first_": {
        "type": "string",
        "name": "q62_personTo62[first]",
        "description": "Emergency Contact, First Name"
      },
      "q62_personTo62_last_": {
        "type": "string",
        "name": "q62_personTo62[last]",
        "description": "Last Name"
      },
      "q46_relationship46": {
        "type": "string",
        "name": "q46_relationship46",
        "description": "Relationship, ex. Father, Mother, etc."
      },
      "q63_addressOf63_addr_line1_": {
        "type": "string",
        "name": "q63_addressOf63[addr_line1]",
        "description": "Address Of Person To Contact In Case Of Emergency, Street Address"
      },
      "q63_addressOf63_addr_line2_": {
        "type": "string",
        "name": "q63_addressOf63[addr_line2]",
        "description": "Street Address Line 2"
      },
      "q63_addressOf63_city_": {
        "type": "string",
        "name": "q63_addressOf63[city]",
        "description": "City"
      },
      "q63_addressOf63_state_": {
        "type": "string",
        "name": "q63_addressOf63[state]",
        "description": "State / Province"
      },
      "q63_addressOf63_postal_": {
        "type": "string",
        "name": "q63_addressOf63[postal]",
        "description": "Postal / Zip Code"
      },
      "q64_contactDetails64_full_": {
        "type": "string",
        "name": "q64_contactDetails64[full]",
        "description": "Phone Number Of Person To Contact In Case Of Emergency, Please enter a valid phone number. (000) 000-0000"
      },
      "q68_emailOf": {
        "type": "string",
        "name": "q68_emailOf",
        "description": "Email Of Person To Contact In Case Of Emergency, example@example.com"
      },
      "q38_highSchool": {
        "type": "string",
        "name": "q38_highSchool",
        "description": "High School Name"
      },
      "q57_otherExtra": {
        "type": "string",
        "name": "q57_otherExtra",
        "description": "Other Extra Curricular Activities"
      },
      "q51_whowhatIs": {
        "type": "string",
        "name": "q51_whowhatIs",
        "description": "Who/what is the name of your sponsor?"
      },
      "q55_ifYes55": {
        "type": "string",
        "name": "q55_ifYes55",
        "description": "To What Course?"
      },
      "q52_whatCourse": {
        "type": "string",
        "name": "q52_whatCourse",
        "description": "What Course Is Your First Choice In The University?"
      },
      "q26_extraCurricular__": {
        "name": "q26_extraCurricular[]",
        "type": "array",
        "description": "Extra Curricular Participation",
        "uniqueItems": true,
        "items": {
          "oneOf": [
            {
              "const": "Student Council",
              "id": "input_26_0",
              "title": "Student Council"
            },
            {
              "const": "Class Officer",
              "id": "input_26_1",
              "title": "Class Officer"
            },
            {
              "const": "Club/Organization",
              "id": "input_26_2",
              "title": "Club/Organization"
            },
            {
              "const": "Varsity Player",
              "id": "input_26_3",
              "title": "Varsity Player"
            },
            {
              "const": "Others (please specify below)",
              "id": "input_26_4",
              "title": "Others (please specify below)"
            }
          ]
        }
      },
      "q56_resourcesAvailable56__": {
        "name": "q56_resourcesAvailable56[]",
        "type": "array",
        "description": "Resources Available",
        "uniqueItems": true,
        "items": {
          "oneOf": [
            {
              "const": "Car/Transportation",
              "id": "input_56_0",
              "title": "Car/Transportation"
            },
            {
              "const": "Event Venue/House",
              "id": "input_56_1",
              "title": "Event Venue/House"
            },
            {
              "const": "Printer",
              "id": "input_56_2",
              "title": "Printer"
            },
            {
              "const": "LCD Projector",
              "id": "input_56_3",
              "title": "LCD Projector"
            },
            {
              "const": "Laptop",
              "id": "input_56_4",
              "title": "Laptop"
            }
          ]
        }
      },
      "q25_skillstalents__": {
        "name": "q25_skillstalents[]",
        "type": "array",
        "description": "Skills/Talents",
        "uniqueItems": true,
        "items": {
          "oneOf": [
            {
              "const": "Acting",
              "id": "input_25_0",
              "title": "Acting"
            },
            {
              "const": "Arts/Crafts",
              "id": "input_25_1",
              "title": "Arts/Crafts"
            },
            {
              "const": "Calculating",
              "id": "input_25_2",
              "title": "Calculating"
            },
            {
              "const": "Dancing",
              "id": "input_25_3",
              "title": "Dancing"
            },
            {
              "const": "Debating",
              "id": "input_25_4",
              "title": "Debating"
            },
            {
              "const": "Drawing",
              "id": "input_25_5",
              "title": "Drawing"
            },
            {
              "const": "Eating",
              "id": "input_25_6",
              "title": "Eating"
            },
            {
              "const": "Fashion",
              "id": "input_25_7",
              "title": "Fashion"
            },
            {
              "const": "Photography",
              "id": "input_25_8",
              "title": "Photography"
            },
            {
              "const": "Playing drums",
              "id": "input_25_9",
              "title": "Playing drums"
            },
            {
              "const": "Playing guitar",
              "id": "input_25_10",
              "title": "Playing guitar"
            },
            {
              "const": "Playing piano",
              "id": "input_25_11",
              "title": "Playing piano"
            },
            {
              "const": "Programming",
              "id": "input_25_12",
              "title": "Programming"
            },
            {
              "const": "Singing",
              "id": "input_25_13",
              "title": "Singing"
            },
            {
              "const": "Writing",
              "id": "input_25_14",
              "title": "Writing"
            },
            {
              "const": "Others (specify below)",
              "id": "input_25_15",
              "title": "Others (specify below)"
            }
          ]
        }
      },
      "q28_sports__": {
        "name": "q28_sports[]",
        "type": "array",
        "description": "Sports",
        "uniqueItems": true,
        "items": {
          "oneOf": [
            {
              "const": "Badminton",
              "id": "input_28_0",
              "title": "Badminton"
            },
            {
              "const": "Basketball",
              "id": "input_28_1",
              "title": "Basketball"
            },
            {
              "const": "Bowling",
              "id": "input_28_2",
              "title": "Bowling"
            },
            {
              "const": "Chess",
              "id": "input_28_3",
              "title": "Chess"
            },
            {
              "const": "Frisbee",
              "id": "input_28_4",
              "title": "Frisbee"
            },
            {
              "const": "Scrabble",
              "id": "input_28_5",
              "title": "Scrabble"
            },
            {
              "const": "Soccer",
              "id": "input_28_6",
              "title": "Soccer"
            },
            {
              "const": "Softball",
              "id": "input_28_7",
              "title": "Softball"
            },
            {
              "const": "Swimming",
              "id": "input_28_8",
              "title": "Swimming"
            },
            {
              "const": "Table Tennis",
              "id": "input_28_9",
              "title": "Table Tennis"
            },
            {
              "const": "Track and Field",
              "id": "input_28_10",
              "title": "Track and Field"
            },
            {
              "const": "Tennis",
              "id": "input_28_11",
              "title": "Tennis"
            },
            {
              "const": "Volleyball",
              "id": "input_28_12",
              "title": "Volleyball"
            },
            {
              "const": "Others (specify below)",
              "id": "input_28_13",
              "title": "Others (specify below)"
            }
          ]
        }
      },
      "q50_doYou": {
        "name": "q50_doYou",
        "type": "string",
        "description": "Do You Have Any Scholarship?",
        "enum": [
          "Yes",
          "No"
        ]
      },
      "q54_doYou54": {
        "name": "q54_doYou54",
        "type": "string",
        "description": "Do You Plan To Shift To Another Course?",
        "enum": [
          "Yes",
          "No"
        ]
      },
      "q8_yearLevel8": {
        "type": "string",
        "enum": [
          "",
          "1st Year",
          "2nd Year",
          "3rd Year",
          "4th Year",
          "5th Year"
        ],
        "description": "Year Level, for Incoming A.Y. '12-'13 undefined",
        "name": "q8_yearLevel8"
      },
      "q59_degreeProgram": {
        "type": "string",
        "enum": [
          "",
          "BS in Pre-Med Physics",
          "BS in Physics minor in Economics",
          "BS in Physics minor in Finance",
          "BS in Physics with specialization in Material Science",
          "BS in Physics with specialization in Medical Instrumentation",
          "MS in Physics",
          "PhD in Physics"
        ],
        "description": "Degree Program undefined",
        "name": "q59_degreeProgram"
      },
      "q47_networkProvider47": {
        "type": "string",
        "enum": [
          "",
          "Globe",
          "Red Mobile",
          "Smart",
          "Sun Cellular",
          "Talk N' Text",
          "Touch Mobile (TM)"
        ],
        "description": "Network Provider undefined",
        "name": "q47_networkProvider47"
      },
      "q15_birthDate_month_": {
        "type": "string",
        "enum": [
          "",
          "1",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9",
          "10",
          "11",
          "12"
        ],
        "description": "Month undefined",
        "name": "q15_birthDate[month]"
      },
      "q15_birthDate_day_": {
        "type": "string",
        "enum": [
          "",
          "1",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9",
          "10",
          "11",
          "12",
          "13",
          "14",
          "15",
          "16",
          "17",
          "18",
          "19",
          "20",
          "21",
          "22",
          "23",
          "24",
          "25",
          "26",
          "27",
          "28",
          "29",
          "30",
          "31"
        ],
        "description": "Day undefined",
        "name": "q15_birthDate[day]"
      },
      "q15_birthDate_year_": {
        "type": "string",
        "enum": [
          "",
          "2024",
          "2023",
          "2022",
          "2021",
          "2020",
          "2019",
          "2018",
          "2017",
          "2016",
          "2015",
          "2014",
          "2013",
          "2012",
          "2011",
          "2010",
          "2009",
          "2008",
          "2007",
          "2006",
          "2005",
          "2004",
          "2003",
          "2002",
          "2001",
          "2000",
          "1999",
          "1998",
          "1997",
          "1996",
          "1995",
          "1994",
          "1993",
          "1992",
          "1991",
          "1990",
          "1989",
          "1988",
          "1987",
          "1986",
          "1985",
          "1984",
          "1983",
          "1982",
          "1981",
          "1980",
          "1979",
          "1978",
          "1977",
          "1976",
          "1975",
          "1974",
          "1973",
          "1972",
          "1971",
          "1970",
          "1969",
          "1968",
          "1967",
          "1966",
          "1965",
          "1964",
          "1963",
          "1962",
          "1961",
          "1960",
          "1959",
          "1958",
          "1957",
          "1956",
          "1955",
          "1954",
          "1953",
          "1952",
          "1951",
          "1950",
          "1949",
          "1948",
          "1947",
          "1946",
          "1945",
          "1944",
          "1943",
          "1942",
          "1941",
          "1940",
          "1939",
          "1938",
          "1937",
          "1936",
          "1935",
          "1934",
          "1933",
          "1932",
          "1931",
          "1930",
          "1929",
          "1928",
          "1927",
          "1926",
          "1925",
          "1924",
          "1923",
          "1922",
          "1921",
          "1920"
        ],
        "description": "Year undefined",
        "name": "q15_birthDate[year]"
      },
      "q44_presentAddress44_country_": {
        "type": "string",
        "enum": [
          "",
          "Afghanistan",
          "Albania",
          "Algeria",
          "American Samoa",
          "Andorra",
          "Angola",
          "Anguilla",
          "Antigua and Barbuda",
          "Argentina",
          "Armenia",
          "Aruba",
          "Australia",
          "Austria",
          "Azerbaijan",
          "The Bahamas",
          "Bahrain",
          "Bangladesh",
          "Barbados",
          "Belarus",
          "Belgium",
          "Belize",
          "Benin",
          "Bermuda",
          "Bhutan",
          "Bolivia",
          "Bosnia and Herzegovina",
          "Botswana",
          "Brazil",
          "Brunei",
          "Bulgaria",
          "Burkina Faso",
          "Burundi",
          "Cambodia",
          "Cameroon",
          "Canada",
          "Cape Verde",
          "Cayman Islands",
          "Central African Republic",
          "Chad",
          "Chile",
          "China",
          "Christmas Island",
          "Cocos (Keeling) Islands",
          "Colombia",
          "Comoros",
          "Congo",
          "Cook Islands",
          "Costa Rica",
          "Cote d'Ivoire",
          "Croatia",
          "Cuba",
          "Curaçao",
          "Cyprus",
          "Czech Republic",
          "Democratic Republic of the Congo",
          "Denmark",
          "Djibouti",
          "Dominica",
          "Dominican Republic",
          "Ecuador",
          "Egypt",
          "El Salvador",
          "Equatorial Guinea",
          "Eritrea",
          "Estonia",
          "Ethiopia",
          "Falkland Islands",
          "Faroe Islands",
          "Fiji",
          "Finland",
          "France",
          "French Polynesia",
          "Gabon",
          "The Gambia",
          "Georgia",
          "Germany",
          "Ghana",
          "Gibraltar",
          "Greece",
          "Greenland",
          "Grenada",
          "Guadeloupe",
          "Guam",
          "Guatemala",
          "Guernsey",
          "Guinea",
          "Guinea-Bissau",
          "Guyana",
          "Haiti",
          "Honduras",
          "Hong Kong",
          "Hungary",
          "Iceland",
          "India",
          "Indonesia",
          "Iran",
          "Iraq",
          "Ireland",
          "Israel",
          "Italy",
          "Jamaica",
          "Japan",
          "Jersey",
          "Jordan",
          "Kazakhstan",
          "Kenya",
          "Kiribati",
          "North Korea",
          "South Korea",
          "Kosovo",
          "Kuwait",
          "Kyrgyzstan",
          "Laos",
          "Latvia",
          "Lebanon",
          "Lesotho",
          "Liberia",
          "Libya",
          "Liechtenstein",
          "Lithuania",
          "Luxembourg",
          "Macau",
          "Macedonia",
          "Madagascar",
          "Malawi",
          "Malaysia",
          "Maldives",
          "Mali",
          "Malta",
          "Marshall Islands",
          "Martinique",
          "Mauritania",
          "Mauritius",
          "Mayotte",
          "Mexico",
          "Micronesia",
          "Moldova",
          "Monaco",
          "Mongolia",
          "Montenegro",
          "Montserrat",
          "Morocco",
          "Mozambique",
          "Myanmar",
          "Nagorno-Karabakh",
          "Namibia",
          "Nauru",
          "Nepal",
          "Netherlands",
          "Netherlands Antilles",
          "New Caledonia",
          "New Zealand",
          "Nicaragua",
          "Niger",
          "Nigeria",
          "Niue",
          "Norfolk Island",
          "Turkish Republic of Northern Cyprus",
          "Northern Mariana",
          "Norway",
          "Oman",
          "Pakistan",
          "Palau",
          "Palestine",
          "Panama",
          "Papua New Guinea",
          "Paraguay",
          "Peru",
          "Philippines",
          "Pitcairn Islands",
          "Poland",
          "Portugal",
          "Puerto Rico",
          "Qatar",
          "Republic of the Congo",
          "Romania",
          "Russia",
          "Rwanda",
          "Saint Barthelemy",
          "Saint Helena",
          "Saint Kitts and Nevis",
          "Saint Lucia",
          "Saint Martin",
          "Saint Pierre and Miquelon",
          "Saint Vincent and the Grenadines",
          "Samoa",
          "San Marino",
          "Sao Tome and Principe",
          "Saudi Arabia",
          "Senegal",
          "Serbia",
          "Seychelles",
          "Sierra Leone",
          "Singapore",
          "Slovakia",
          "Slovenia",
          "Solomon Islands",
          "Somalia",
          "Somaliland",
          "South Africa",
          "South Ossetia",
          "South Sudan",
          "Spain",
          "Sri Lanka",
          "Sudan",
          "Suriname",
          "Svalbard",
          "eSwatini",
          "Sweden",
          "Switzerland",
          "Syria",
          "Taiwan",
          "Tajikistan",
          "Tanzania",
          "Thailand",
          "Timor-Leste",
          "Togo",
          "Tokelau",
          "Tonga",
          "Transnistria Pridnestrovie",
          "Trinidad and Tobago",
          "Tristan da Cunha",
          "Tunisia",
          "Turkey",
          "Turkmenistan",
          "Turks and Caicos Islands",
          "Tuvalu",
          "Uganda",
          "Ukraine",
          "United Arab Emirates",
          "United Kingdom",
          "United States",
          "Uruguay",
          "Uzbekistan",
          "Vanuatu",
          "Vatican City",
          "Venezuela",
          "Vietnam",
          "British Virgin Islands",
          "Isle of Man",
          "US Virgin Islands",
          "Wallis and Futuna",
          "Western Sahara",
          "Yemen",
          "Zambia",
          "Zimbabwe",
          "other"
        ],
        "description": "Country undefined",
        "name": "q44_presentAddress44[country]"
      },
      "q45_permanentAddress45_country_": {
        "type": "string",
        "enum": [
          "",
          "Afghanistan",
          "Albania",
          "Algeria",
          "American Samoa",
          "Andorra",
          "Angola",
          "Anguilla",
          "Antigua and Barbuda",
          "Argentina",
          "Armenia",
          "Aruba",
          "Australia",
          "Austria",
          "Azerbaijan",
          "The Bahamas",
          "Bahrain",
          "Bangladesh",
          "Barbados",
          "Belarus",
          "Belgium",
          "Belize",
          "Benin",
          "Bermuda",
          "Bhutan",
          "Bolivia",
          "Bosnia and Herzegovina",
          "Botswana",
          "Brazil",
          "Brunei",
          "Bulgaria",
          "Burkina Faso",
          "Burundi",
          "Cambodia",
          "Cameroon",
          "Canada",
          "Cape Verde",
          "Cayman Islands",
          "Central African Republic",
          "Chad",
          "Chile",
          "China",
          "Christmas Island",
          "Cocos (Keeling) Islands",
          "Colombia",
          "Comoros",
          "Congo",
          "Cook Islands",
          "Costa Rica",
          "Cote d'Ivoire",
          "Croatia",
          "Cuba",
          "Curaçao",
          "Cyprus",
          "Czech Republic",
          "Democratic Republic of the Congo",
          "Denmark",
          "Djibouti",
          "Dominica",
          "Dominican Republic",
          "Ecuador",
          "Egypt",
          "El Salvador",
          "Equatorial Guinea",
          "Eritrea",
          "Estonia",
          "Ethiopia",
          "Falkland Islands",
          "Faroe Islands",
          "Fiji",
          "Finland",
          "France",
          "French Polynesia",
          "Gabon",
          "The Gambia",
          "Georgia",
          "Germany",
          "Ghana",
          "Gibraltar",
          "Greece",
          "Greenland",
          "Grenada",
          "Guadeloupe",
          "Guam",
          "Guatemala",
          "Guernsey",
          "Guinea",
          "Guinea-Bissau",
          "Guyana",
          "Haiti",
          "Honduras",
          "Hong Kong",
          "Hungary",
          "Iceland",
          "India",
          "Indonesia",
          "Iran",
          "Iraq",
          "Ireland",
          "Israel",
          "Italy",
          "Jamaica",
          "Japan",
          "Jersey",
          "Jordan",
          "Kazakhstan",
          "Kenya",
          "Kiribati",
          "North Korea",
          "South Korea",
          "Kosovo",
          "Kuwait",
          "Kyrgyzstan",
          "Laos",
          "Latvia",
          "Lebanon",
          "Lesotho",
          "Liberia",
          "Libya",
          "Liechtenstein",
          "Lithuania",
          "Luxembourg",
          "Macau",
          "Macedonia",
          "Madagascar",
          "Malawi",
          "Malaysia",
          "Maldives",
          "Mali",
          "Malta",
          "Marshall Islands",
          "Martinique",
          "Mauritania",
          "Mauritius",
          "Mayotte",
          "Mexico",
          "Micronesia",
          "Moldova",
          "Monaco",
          "Mongolia",
          "Montenegro",
          "Montserrat",
          "Morocco",
          "Mozambique",
          "Myanmar",
          "Nagorno-Karabakh",
          "Namibia",
          "Nauru",
          "Nepal",
          "Netherlands",
          "Netherlands Antilles",
          "New Caledonia",
          "New Zealand",
          "Nicaragua",
          "Niger",
          "Nigeria",
          "Niue",
          "Norfolk Island",
          "Turkish Republic of Northern Cyprus",
          "Northern Mariana",
          "Norway",
          "Oman",
          "Pakistan",
          "Palau",
          "Palestine",
          "Panama",
          "Papua New Guinea",
          "Paraguay",
          "Peru",
          "Philippines",
          "Pitcairn Islands",
          "Poland",
          "Portugal",
          "Puerto Rico",
          "Qatar",
          "Republic of the Congo",
          "Romania",
          "Russia",
          "Rwanda",
          "Saint Barthelemy",
          "Saint Helena",
          "Saint Kitts and Nevis",
          "Saint Lucia",
          "Saint Martin",
          "Saint Pierre and Miquelon",
          "Saint Vincent and the Grenadines",
          "Samoa",
          "San Marino",
          "Sao Tome and Principe",
          "Saudi Arabia",
          "Senegal",
          "Serbia",
          "Seychelles",
          "Sierra Leone",
          "Singapore",
          "Slovakia",
          "Slovenia",
          "Solomon Islands",
          "Somalia",
          "Somaliland",
          "South Africa",
          "South Ossetia",
          "South Sudan",
          "Spain",
          "Sri Lanka",
          "Sudan",
          "Suriname",
          "Svalbard",
          "eSwatini",
          "Sweden",
          "Switzerland",
          "Syria",
          "Taiwan",
          "Tajikistan",
          "Tanzania",
          "Thailand",
          "Timor-Leste",
          "Togo",
          "Tokelau",
          "Tonga",
          "Transnistria Pridnestrovie",
          "Trinidad and Tobago",
          "Tristan da Cunha",
          "Tunisia",
          "Turkey",
          "Turkmenistan",
          "Turks and Caicos Islands",
          "Tuvalu",
          "Uganda",
          "Ukraine",
          "United Arab Emirates",
          "United Kingdom",
          "United States",
          "Uruguay",
          "Uzbekistan",
          "Vanuatu",
          "Vatican City",
          "Venezuela",
          "Vietnam",
          "British Virgin Islands",
          "Isle of Man",
          "US Virgin Islands",
          "Wallis and Futuna",
          "Western Sahara",
          "Yemen",
          "Zambia",
          "Zimbabwe",
          "other"
        ],
        "description": "Country undefined",
        "name": "q45_permanentAddress45[country]"
      },
      "q49_estimateYour": {
        "type": "string",
        "enum": [
          "",
          "P20,000 and below",
          "P21,000 - P40,000",
          "P41,000 - P60,000",
          "P61,000 - P80,000",
          "P81,000 and above"
        ],
        "description": "Estimate Your Family Monthly Income undefined",
        "name": "q49_estimateYour"
      },
      "q53_whatIs53": {
        "type": "string",
        "enum": [
          "",
          "Take another Bachelor's Degree",
          "Take a Master's Degree",
          "Take a Master's Degree, then PhD",
          "Study in Graduate School of Medicine",
          "Study in Graduate School of Law",
          "Work"
        ],
        "description": "What Is Your Plan After College? undefined",
        "name": "q53_whatIs53"
      },
      "q69_interests": {
        "type": "string",
        "description": "Interests",
        "name": "q69_interests"
      },
      "q48_suggestions48": {
        "type": "string",
        "description": "Suggestions / Comments",
        "name": "q48_suggestions48"
      },
      "g-recaptcha-response": {
        "type": "string",
        "description": "",
        "name": "g-recaptcha-response"
      },
      "h-captcha-response": {
        "type": "string",
        "description": "",
        "name": "h-captcha-response"
      },
      "_data": {
        "type": "string",
        "description": "Your data",
        "name": "_data"
      }
    }
  }
}